##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[]


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Upload the video and print a confirmation.
video_file_name = "/content/drive/MyDrive/法学课程/刑事执行法1-1-1_x264.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/ubym0880634x
.........

## Call `generate_content`

In [9]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

# Create the prompt.
prompt = """角色设定: 你是一个学生，正在学习刑事执行，想要做到能够清晰、准确地理解法律文本，并能结合实际案例进行分析。
任务描述: 请观看我下面给出的刑事执行法的课程视频。在观看完视频后，请回答下列问题。确保你的回答详细、准确，并结合视频中的相关信息来支持你的答案。
问题列表:
1.	管制刑与拘役的主要区别是什么？
2.	管制在我国的使用率是多少？
3.	拘役的刑期范围是多长？
4.	在刑罚结构中，管制刑的作用是什么？
5.	社区矫正制度与管制刑有什么关系？
6.	拘役的执行主体是什么？
7.	短期自由刑的一个主要优势是什么？
8.	在什么情况下，法官可以选择易科罚金？
9.	在我国，拘役的适用情况如何变化？
10.	什么是管制刑的主要批评意见之一？
11.	根据课程内容，为什么有人主张废除管制刑？请分析其理由。
12.	分析拘役的设计初衷是如何反映在其适用上的。
13.	在信息技术时代，管制刑的执行有哪些优势？
14.	请举例说明短期自由刑的弊端如何影响监狱管理。
15.	结合课程内容，讨论为什么短期自由刑在国际上受到审慎对待。
16.	在我国，如何看待短期自由刑与其他非监禁措施的关系？
17.	分析社区矫正的再犯率与管制刑的关系。
18.	探讨短期自由刑替代措施的重要性，以及易科制度的优势。
19.	如何评价当前我国拘役的适用状况与历史背景的关系？
20.	结合课程内容，讨论我国对短期自由刑的未来发展方向。
21.	小李因盗窃罪被判处管制，法院决定给予缓刑的可能性。
分析法院在选择管制与缓刑时需要考虑哪些因素？
如果小李的行为在社区矫正中表现良好，是否应考虑撤销管制？请说明理由。
22.	小张因醉驾被判处拘役3个月，法院决定在看守所执行。
分析拘役在看守所执行可能面临的困难和问题。
如果小张在拘役期间表现良好，能否申请减刑或缓刑？请结合相关法律规定进行分析。
23.	小王因轻微犯罪被判处拘役，但由于经济困难，无法支付罚金。
讨论易科罚金制度在此情境中的适用性，如何避免短期自由刑的弊端？
如果小王选择易科劳役，他的社会责任感会如何受到影响？
24.	某地区刑罚改革过程中，讨论是否保留管制刑。
从历史背景出发，分析管制刑的存在是否仍然具有现实意义？
如果该地区人口流动性大，基层组织能力下降，这将如何影响管制刑的实施效果？
25.	小赵因轻罪被判处管制，并且参加了社区矫正。
结合课程内容，分析社区矫正如何降低再犯率，管制刑的作用是什么？
请你基于视频内容，回答问题，回答格式是先列出问题，再给出答案。对于在视频中无法推断出答案的问题，请回答视频并未提及有关的知识点，无法作答。
"""
contents = [video_file, prompt]

print("Making LLM inference request...")
response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

Making LLM inference request...


你好。以下是对你的问题列表的回答：

1. 管制刑与拘役的主要区别是什么？

管制和拘役的主要区别在于刑期长短、适用对象和执行场所。管制刑期较短，适用于未成年犯罪、过失犯罪或初次犯罪，并且犯罪性质不严重、主观恶性不大的罪犯。被判处管制刑的罪犯在一定程度上仍可以保持人身自由，可以从事工作或继续学业，但需要定期向社区矫正机构汇报。

拘役刑期较长，通常在看守所执行。被判处拘役的罪犯被剥夺人身自由，需要在看守所进行劳动改造。拘役的执行主体是公安机关。

2. 管制在我国的使用率是多少？

视频中老师提到管制在2019年的适用率是0.018，也就是千分之1.8。

3. 拘役的刑期范围是多长？

拘役的刑期为1个月以上6个月以下。

4. 在刑罚结构中，管制刑的作用是什么？

管制刑的主要作用在于限制犯罪人的自由，加强对其行为的监管，并且通过社区矫正和社会服务等方式，促使其进行劳动改造，提高其悔改程度，降低再犯率。

5. 社区矫正制度与管制刑有什么关系？

社区矫正制度与管制刑密切相关。被判处管制刑的罪犯一般都需要参加社区矫正，定期向社区矫正机构汇报思想改造情况，并且接受社区矫正机构的监督。

6. 拘役的执行主体是什么？

拘役的执行主体是公安机关。

7. 短期自由刑的一个主要优势是什么？

短期自由刑的一个主要优势是时间较短，对犯罪者的生活影响相对较小，可以使他们尽快回到正常的生活轨道中。

8. 在什么情况下，法官可以选择易科罚金？

视频中并未提及具体哪些情况可以选择易科罚金。但是，易科制度的适用旨在避免短期自由刑的弊端，例如短期自由刑可能使罪犯感染恶习，不利于其改造。

9. 在我国，拘役的适用情况如何变化？

视频中老师根据数据分析显示，近年来越来越多的轻罪被判处拘役。

10. 什么是管制刑的主要批评意见之一？

管制刑的主要批评意见之一是其刑罚力度太轻，缺乏威慑力，不足以震慑罪犯。

11. 根据课程内容，为什么有人主张废除管制刑？请分析其理由。

视频中并未提及主张废除管制刑的理由。

12. 分析拘役的设计初衷是如何反映在其适用上的。

拘役的设计初衷是通过短期监禁和劳动改造，使罪犯认识到错误，提高悔改程度。视频中展示了拘役在看守所的执行情况，例如罪犯在看守所的日常活动和生活环境。这些都体现了拘役的改造功能。

13. 在信息技术时代，管制刑的执行有哪些优势？

视频中并未提及在信息技术时代，管制刑的执行有哪些优势。

14. 请举例说明短期自由刑的弊端如何影响监狱管理。

视频中提到短期自由刑的弊端之一是可能使罪犯感染恶习，这会增加监狱的管理难度。例如，拘役的执行场所通常是看守所，看守所的人口密度大，管理空间狭小，罪犯之间容易相互影响，可能会学习到更多的犯罪手段和技巧。

15. 结合课程内容，讨论为什么短期自由刑在国际上受到审慎对待。

短期自由刑在国际上受到审慎对待，是因为其弊端大于优势。短期自由刑时间短，难以真正教育和改造罪犯，反而可能使他们沾染更多恶习，增加社会危害性。

16. 在我国，如何看待短期自由刑与其他非监禁措施的关系？

视频中提到，短期自由刑和罚金、社区矫正可以互相转换。换言之，在轻微案件中，法官可以根据具体情况选择判决短期自由刑、罚金或社区矫正。

17. 分析社区矫正的再犯率与管制刑的关系。

视频中提到，社区矫正的再犯率很低，约为千分之二。但由于管制刑判决数量少，再犯率与管制刑判决数存在一定关系，但无法明确社区矫正的再犯率与管制刑的直接关系。

18. 探讨短期自由刑替代措施的重要性，以及易科制度的优势。

短期自由刑的替代措施，例如罚金和社区矫正，可以避免短期监禁带来的弊端，例如感染恶习和增加管理难度。易科制度的优势在于，它可以根据罪犯的经济状况和悔改表现，将短期自由刑转换为罚金或社区矫正，使刑罚更加人性化。

19. 如何评价当前我国拘役的适用状况与历史背景的关系？

视频中提到，拘役的适用与我国的社会历史背景相关，最初拘役是在战争时期所制定的，在现代社会与最初使用拘役的时期已经发生了巨大的变化。

20. 结合课程内容，讨论我国对短期自由刑的未来发展方向。

视频中未提到短期自由刑未来发展方向。

21. 小李因盗窃罪被判处管制，法院决定给予缓刑的可能性。分析法院在选择管制与缓刑时需要考虑哪些因素？如果小李的行为在社区矫正中表现良好，是否应考虑撤销管制？请说明理由。

小李被判处管制时，法院并未直接讨论缓刑的可能性。法院在选择管制与缓刑时，需要考虑罪犯的犯罪情节、主观恶性、人身危险性、社会危害性以及社区矫正的执行情况等因素。如果小李在社区矫正中表现良好，并且不再构成社会危害，法院可以考虑撤销管制。

22. 小张因醉驾被判处拘役3个月，法院决定在看守所执行。分析拘役在看守所执行可能面临的困难和问题。如果小张在拘役期间表现良好，能否申请减刑或缓刑？请结合相关法律规定进行分析。

拘役在看守所执行可能面临的困难和问题包括：看守所的环境比较恶劣，而且人均空间非常狭小，执行的实际改造效果不理想。
根据我国刑法的相关规定，小张因醉驾被判处拘役，如果他在服刑期间表现良好，符合相关法律规定的条件，是可以申请减刑的，但一般不能申请缓刑。

23. 小王因轻微犯罪被判处拘役，但由于经济困难，无法支付罚金。讨论易科罚金制度在此情境中的适用性，如何避免短期自由刑的弊端？如果小王选择易科劳役，他的社会责任感会如何受到影响？

视频中并未提及小王因轻微犯罪被判处拘役是否可以选择易科罚金。此外，视频未提及小王选择易科劳役，其社会责任感会如何受到影响。

24. 某地区刑罚改革过程中，讨论是否保留管制刑。从历史背景出发，分析管制刑的存在是否仍然具有现实意义？如果该地区人口流动性大，基层组织能力下降，这将如何影响管制刑的实施效果？

管制刑最初是在战争时期，为了加强对战犯改造而设立的。在现代社会，管制刑的存在仍然具有现实意义，因为它可以限制犯罪人的自由，加强对其行为的监管，并通过社区矫正等方式，促使其悔改，降低再犯率。

如果该地区人口流动性大，基层组织能力下降，管制刑的实施效果可能会受到影响。例如，人口流动性大可能会导致难以对罪犯进行有效的监督，基层组织能力下降可能会导致社区矫正工作难以开展。

希望这些信息对你有帮助。

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")